In [3]:
! pip install -q langchain_community tiktoken langchain-openai chromadb langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.4 MB/s eta 0:00:

In [4]:
!pip install -q pypdf gradio

### Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Libraries and Enter API Keys

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''  # Your LangSmith key
os.environ['OPENAI_API_KEY'] = ''  # Your OpenRouter/OpenAI API key
os.environ['USER_AGENT'] = 'my-langchain-app/1.0'  # Suppress USER_AGENT warning

### Load, Embed Files, and Create the Chain with LangChain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

try:
    # Load and chunk contents of the Persian PDF
    loader = PyPDFLoader(
        file_path="/content/drive/your_file_path.pdf",
        extract_images=False
    )
    docs = loader.load()
    print(f"Loaded {len(docs)} documents")

except Exception as e:
    print(f"Error loading documents: {e}")
    raise

# Split
try:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    splits = text_splitter.split_documents(docs)
    print(f"Created {len(splits)} splits")

except Exception as e:
    print(f"Error splitting documents: {e}")
    raise

# Embed (using multilingual embeddings for Persian)
try:
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )
    retriever = vectorstore.as_retriever()
except Exception as e:
    print(f"Error in embedding: {e}")
    raise

# Custom Persian prompt template
try:
    prompt_template = PromptTemplate.from_template(
        "لطفاً به سؤالم بر اساس متن زیر پاسخ دهید:\n\n{context}\n\nسؤال: {question}\n\nپاسخ به فارسی بنویسید."
    )
except Exception as e:
    print(f"Error creating prompt: {e}")
    raise

# LLM (using OpenRouter's meta-llama/llama-3.2-1b-instruct:free)
try:
    llm = ChatOpenAI(
        model_name="deepseek/deepseek-r1-distill-llama-70b:free",
        temperature=0,
        base_url="https://openrouter.ai/api/v1",
        api_key=os.environ['OPENAI_API_KEY']
    )
except Exception as e:
    print(f"Error initializing LLM: {e}")
    raise

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
try:
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_template
        | llm
        | StrOutputParser()
    )
except Exception as e:
    print(f"Error building chain: {e}")
    raise

Loaded 69 documents
Created 179 splits


/tmp/ipython-input-4287614031.py:41: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Test the LLM directly

In [ ]:
try:
    test_response = llm.invoke("سلام، آیا می‌توانید پاسخ دهید؟")
    print("LLM Test Response:", test_response.content)
except Exception as e:
    print(f"LLM Test Error: {e}")

question = "منظور از انواع شیفت کاری جیست؟ "
# Invoke the chain with a Persian question
try:
    response = rag_chain.invoke(question)
    print("RAG Response:", response)
except Exception as e:
    print(f"Error invoking chain: {e}")
    raise

LLM Test Response: سلام بر شما! بله، میتوانم پاسخ دهم. چگونه میتوانم به شما کمک کنم؟
RAG Response: انواع شیفت کاری به دو دسته اصلی تقسیم میشوند:

### 1. شیفت ثابت:
شیفت ثابت به حالتی اطلاق میشود که ساعات کاری فرد در طول هفته یا ماه ثابت و مشخص است. به عنوان مثال، فرد در شیفت صبح، عصر یا شب به صورت دائمی کار میکند و برنامه کاری او تغییر نمیکنند. در این حالت، فرد در یک شیفت خاص قرار دارد و برنامه کاری او تثبیت شده است.

### 2. نوبت کاری (شیفت چرخشی):
نوبت کاری به حالتی اطلاق میشود که فرد در طول هفته یا ماه در شیفتهای مختلفی کار میکند. به عنوان مثال، فرد ممکن است به صورت چرخشی در شیفتهای صبح، عصر و شب کار کند. در این حالت، برنامه کاری فرد به صورت متناوب و چرخشی است. بر اساس قانون کار، کارکنانی که در شیفتهای مختلف کار میکنند، از مزایای نوبت کاری برخوردار میشوند. این مزایا به شرح زیر است:
- **شیفت صبح و عصر:** 11 درصد اضافه بر مزد
- **شیفت صبح، عصر و شب:** 15 درصد اضافه بر مزد
- **شیفت شب و صبح یا عصر و شب:** 22.5 درصد اضافه بر مزد

همچنین، برای کار شب (از ساعت 22 تا 6 بامداد) 35 درصد اضافه

### Build the UI App with Gradio

In [ ]:
css = """
/* Font and RTL for all elements */
:root, html, body, .gradio-container, .prose, .gr-markdown,
.gr-input, .gr-text-input, .gr-textbox, .gr-radio, .gr-button,
.gr-number, .gr-dropdown, .gr-slider, label, input, textarea, select, button {
  font-family: 'Vazirmatn', sans-serif !important;
  direction: rtl !important;
  text-align: right !important;
}

/* RTL for the whole UI */
html, body, .gradio-container {
  direction: rtl;
  text-align: right;
}

/* App background & spacing with scrollbar prevention */
.gradio-container {
  overflow-x: hidden !important;
  overflow-y: hidden !important;
  background: linear-gradient(135deg, #4F46E5, #800020);
  min-height: 100vh;
  margin: 0;
  padding: 20px;
}

/* Primary "Start" button styling */
#generate-btn {
  background-color: #4F46E5;
  color: white;
  font-size: 20px;
  font-weight: bold;
  border-radius: 12px;
  padding: 14px 28px;
  border: none;
  margin: 20px auto;
  display: block;
  width: fit-content;
  transition: background 0.3s ease;
}
#generate-btn:hover {
  background-color: #800020;
  cursor: pointer;
}

/* Optional error banner */
#error-msg {
  color: white;
  font-weight: bold;
  text-align: center;
  margin-top: 10px;
  background-color: #800020;
  padding: 10px;
  border-radius: 10px;
}


/* Fallback for older radio markup */
input[type="radio"]:checked {
  background-color: #800020 !important;
  border-color: #800020 !important;
  color: white !important;
}

/* Dropdown styling */
.gr-dropdown, .gr-dropdown * {
  font-family: 'Vazirmatn', sans-serif !important;
}
div[role="listbox"], div[role="listbox"] * {
  font-family: 'Vazirmatn', sans-serif !important;
}

/* Ensure font loaded via HTML link */
#question_md, #question_md * {
  direction: rtl !important;
  text-align: right !important;
  font-family: 'Vazirmatn', sans-serif !important;
}

/* Target the Markdown component to prevent horizontal scrollbar */
.gr-markdown {
  overflow-x: hidden !important;
  white-space: normal !important;
  max-width: 100% !important;
  direction: rtl !important;
  text-align: right !important;
  padding: 0 !important;
  margin: 0 !important;
}

/* Target the h1 inside Markdown with higher specificity */
.gradio-container .gr-markdown h1 {
  font-size: 2em !important;
  line-height: 1.2 !important;
  overflow-wrap: break-word !important;
  word-break: break-all !important;
  white-space: normal !important;
  max-width: 100% !important;
  padding: 0 !important;
  margin: 0 !important;
  overflow: hidden !important;
}

/* Force all Markdown wrapper blocks (outer divs) to avoid scrollbars */
div.block.svelte-1svsvh2[dir="ltr"][style*="overflow: auto"] {
  overflow: hidden !important;
}

/* Style for the writer info markdown */
#writer_info {
  right: 1200px !important;
  bottom: 0 !important;
  direction: ltr !important;
  text-align: left !important;
}

"""

In [ ]:
import gradio as gr

# Gradio Function
def ask_question(question):
    """Takes a Persian question, runs RAG chain, and returns the Persian answer."""
    try:
        if not question.strip():
            return "❌ لطفاً یک سؤال وارد کنید."
        response = rag_chain.invoke(question)
        return response
    except Exception as e:
        return f"⚠️ خطا در پردازش سؤال: {e}"

# Custom CSS
css = css

# UI Components
with gr.Blocks(css=css, theme=gr.themes.Soft(primary_hue="indigo")) as demo:
    gr.HTML('<link href="https://fonts.googleapis.com/css2?family=Vazirmatn:wght@400;700&display=swap" rel="stylesheet">')

    gr.Markdown(
        """
        <h1 style='text-align:right; color:white;'>📘 سامانه پرسش و پاسخ مبتنی بر RAG</h1>
        <p style='text-align:right; color:white;'>پرسش خود را دربارهٔ فایل PDF وارد کنید تا مدل به زبان فارسی پاسخ دهد.</p>
        """,
    )

    with gr.Row():
        question_box = gr.Textbox(
            label="سؤال شما:",
            placeholder="مثلاً: منظور از انواع شیفت کاری چیست؟",
            elem_id="question_md",
            lines=2,
            rtl=True,
        )

    # with gr.Row():
    generate_btn = gr.Button("پرسش از مدل", elem_id="generate-btn")

    with gr.Row():
        answer_box = gr.Markdown(
            value="در انتظار پرسش شما...",
            elem_id="answer_md",
        )

    generate_btn.click(fn=ask_question, inputs=question_box, outputs=answer_box)

    gr.Markdown(
        """
        <div id="writer_info">
        <h3 style="font-size:12px; color:white;">طراحی و پیاده‌سازی توسط پریا خلیلی</h3>
        </div>
        """,
    )

# --- Launch the App ---
demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a0cf2b76bd32467f64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
